In [ ]:
import os
import xml.etree.ElementTree as ET

# Poti do map
xml_folder = '/mnt/data'  # Mesto kamor si naložil .xml datoteko
images_folder = '/mnt/data/images/train'  # Slike naj bodo tu
labels_folder = '/mnt/data/labels/train'  # YOLO oznake bomo dali sem

# Poskrbi, da mapa za oznake obstaja
os.makedirs(labels_folder, exist_ok=True)

# Preslikava razredov (po potrebi prilagodi!)
class_mapping = {
    'oseba': 0,
    'avto': 1,
    'kolo': 2
}

# Funkcija za pretvorbo v YOLO
def convert_to_yolo(xml_path, img_folder, label_folder):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    filename = root.find('filename').text
    img_path = os.path.join(img_folder, filename)
    label_path = os.path.join(label_folder, filename.replace('.jpg', '.txt'))

    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    with open(label_path, 'w') as f:
        for obj in root.findall('object'):
            cls = obj.find('name').text
            if cls not in class_mapping:
                continue
            cls_id = class_mapping[cls]
            xml_box = obj.find('bndbox')
            xmin = int(xml_box.find('xmin').text)
            ymin = int(xml_box.find('ymin').text)
            xmax = int(xml_box.find('xmax').text)
            ymax = int(xml_box.find('ymax').text)

            x_center = ((xmin + xmax) / 2) / w
            y_center = ((ymin + ymax) / 2) / h
            width = (xmax - xmin) / w
            height = (ymax - ymin) / h

            f.write(f"{cls_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

# Pretvori en sam XML
convert_to_yolo(os.path.join(xml_folder, 'oznake_20250328_091617.xml'),
                images_folder,
                labels_folder)
